<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

In this Notebook the neighborhoods in Toronto will be analyzed in the same way as the New York city neighborhoods. The notebook consists of four parts corresponding to the required preparational work and the three questions in the assignment:
1. Part 0: Preparations ==> Importing libraries and dependencies
2. Part 1: Creating the dataframe containing the Toronto neighborhoods (Question 1)
3. Part 2: Adding the latitude and longitude to the dataframe (Question 2)
4. Part 3: Explore and cluster the neighborhoods of Toronto (Question 3)

## Table of Contents 

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item0">Part 0: Preparations ==> Import libraries and dependencies</a>

2. <a href="#item1">Part 1: Creating the dataframe containing the Toronto neighborhoods (Question 1)</a>

3. <a href="#item2">Part 2: Adding the latitude and longitude to the dataframe (Question 2)</a>

4. <a href="#item3">Part 3: Explore and cluster the neighborhoods of Toronto (Question 3)</a>
    
</font>
</div>

<a id='item0'></a>

<h2 align center> Part 0: Preparations ==> Import libraries and dependencies

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

<a id='item1'></a>

<h2 align center> Part 1: Creating the dataframe containing the Toronto neighborhoods

The list of boroughs and neighborhoods of Toronto is given on the following Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 
In order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe it will be read with the pandas library, with some cleaning afterwards.

First obtain the data from the website. Pandas returns a list of dataframes, where the first one is the list postcodes for Toronto, i.e. raw_df[0] is the dataframe we need.

In [29]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_df = pd.read_html(url)
toronto_df = raw_df[0]
toronto_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


Now we need to clean the table:
Remove all lines where Boroughs is "Not assigned"

In [30]:
toronto_df = toronto_df[toronto_df.Borough != 'Not assigned']
toronto_df = toronto_df.reset_index(drop=True)
print('Number of boroughs/Neighbourhoods: {}'.format(len(toronto_df.index)))
print('Number of unique postcodes: {}'.format(len(toronto_df['Postcode'].unique())))

Number of boroughs/Neighbourhoods: 210
Number of unique postcodes: 103


Second step in cleaning: For all neighborhoods, where the borough is existing but no neighborhood assigned (i.e. "Not assigned"), set the name of the neighborhood to the name of the Borough. 

In [31]:
sel = toronto_df.Neighbourhood == 'Not assigned'
toronto_df.loc[sel,'Neighbourhood'] = toronto_df[sel].Borough
toronto_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Third step in cleaning the data: For duplicate postcode entries, remove one entry nd set the neighborhood to the two entries.

In [32]:
toronto_df = toronto_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
toronto_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [33]:
print('final shape of dataframe: {}'.format(toronto_df.shape))
print('Number of neighbourhood entries: {}'.format(toronto_df.shape[0]))

final shape of dataframe: (103, 3)
Number of neighbourhood entries: 103


<a id='item2'></a>

<h2 align center> Part 2: Adding the latitude and longitude to the dataframe (Question 2)

Now we have the dataframe with the list of neighbourhoods of Toronto, we still need latitude and longitude for each postcode. These will be added to the dataframe using the geocode library. To take into account the reliability issues of the library, the procedure recommended in the assignment will be used.

First install geocoder package. And then use arcgis instead of google, as google just returns "Request denied".

In [10]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [34]:
import geocoder # import geocoder

# Create dataframe entries for latitude and longitude
toronto_df = toronto_df.assign(Latitude=0)
toronto_df = toronto_df.assign(Longitude=0)

# Limit for trials to get Latitude and longitude
limit_tries = 5

# First loop over dataframe entries containing the postcodes
for i in range(len(toronto_df.index)):
    # initialize your variable to None
    lat_lng_coords = None
    
    searchstring = '{}, Toronto, Ontario'.format(toronto_df.loc[i,'Postcode'])
    print('Searching for ' + searchstring)
    # loop until you get the coordinates or the number of tries exceeds a limit
    tries = 1
    while((lat_lng_coords is None) and (tries <= limit_tries)):
        g = geocoder.arcgis(searchstring)
        lat_lng_coords = g.latlng
        tries +=1
    
    print(g)
    if not(lat_lng_coords is None):
        toronto_df.loc[i,'Latitude'] = lat_lng_coords[0]
        toronto_df.loc[i,'Longitude'] = lat_lng_coords[1]
        

Searching for M1B, Toronto, Ontario
<[OK] Arcgis - Geocode [M1B]>
Searching for M1C, Toronto, Ontario
<[OK] Arcgis - Geocode [M1C]>
Searching for M1E, Toronto, Ontario
<[OK] Arcgis - Geocode [M1E]>
Searching for M1G, Toronto, Ontario
<[OK] Arcgis - Geocode [M1G]>
Searching for M1H, Toronto, Ontario
<[OK] Arcgis - Geocode [M1H]>
Searching for M1J, Toronto, Ontario
<[OK] Arcgis - Geocode [M1J]>
Searching for M1K, Toronto, Ontario
<[OK] Arcgis - Geocode [M1K]>
Searching for M1L, Toronto, Ontario
<[OK] Arcgis - Geocode [M1L]>
Searching for M1M, Toronto, Ontario
<[OK] Arcgis - Geocode [M1M]>
Searching for M1N, Toronto, Ontario
<[OK] Arcgis - Geocode [M1N]>
Searching for M1P, Toronto, Ontario
<[OK] Arcgis - Geocode [M1P]>
Searching for M1R, Toronto, Ontario
<[OK] Arcgis - Geocode [M1R]>
Searching for M1S, Toronto, Ontario
<[OK] Arcgis - Geocode [M1S]>
Searching for M1T, Toronto, Ontario
<[OK] Arcgis - Geocode [M1T]>
Searching for M1V, Toronto, Ontario
<[OK] Arcgis - Geocode [M1V]>
Searching 

Let's take a quick look at the data.

In [35]:
toronto_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696770,-79.259967


#### Use geopy library to get the latitude and longitude values of Toronto

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>tor_explorer</em>, as shown below.

In [36]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [38]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<a id='item3'></a>

<h2 align center> Part 3: Explore and cluster the neighborhoods of Toronto (Question 3)

For the following analysis all neighborhoods of Toronto will be kept.

First repeat the map of Toronto and its neighborhoods.

In [39]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [40]:
CLIENT_ID = 'QNS1YOELBVKBLGWVSQ502U22GD0U5CVRQRZJBEUXVPSGBPKL' # your Foursquare ID
CLIENT_SECRET = 'JWO50SJLNFPC3VIE4GQOW1L1NOZ4HUHLHYM5UDW1KL2U5D02' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QNS1YOELBVKBLGWVSQ502U22GD0U5CVRQRZJBEUXVPSGBPKL
CLIENT_SECRET:JWO50SJLNFPC3VIE4GQOW1L1NOZ4HUHLHYM5UDW1KL2U5D02


#### Define function to obtain nearby venues for all the neighborhoods in Toronto

In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [44]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )



Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

#### Let's check the size of the resulting dataframe

In [45]:
print(toronto_venues.shape)
toronto_venues.head()

(2437, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
2,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.765815,-79.175193,Canadian Carpet Cleaning and Janitorial Service,43.767731,-79.174240,Dry Cleaner
4,"Guildwood,Morningside,West Hill",43.765815,-79.175193,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping


Let's check how many venues were returned for each neighborhood

In [46]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,14,14,14,14,14,14
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",15,15,15,15,15,15
"Alderwood,Long Branch",4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",22,22,22,22,22,22
Berczy Park,61,61,61,61,61,61
"Birch Cliff,Cliffside West",6,6,6,6,6,6


#### Let's find out how many unique categories can be curated from all the returned venues

In [47]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 261 uniques categories.


There is one category returned called "Neighborhood". This will colide in the one-hot encoding with the column neighborhood. This will be changed by replacing those entries with "Neighborhood_venue".

In [58]:
toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
315,The Beaches,43.676531,-79.295425,Upper Beaches,43.680563,-79.292869,Neighborhood
388,Studio District,43.660629,-79.334855,Leslieville,43.662070,-79.337856,Neighborhood
926,Central Bay Street,43.656091,-79.384930,Downtown Toronto,43.653232,-79.385296,Neighborhood
1080,"Adelaide,King,Richmond",43.649700,-79.382582,Downtown Toronto,43.653232,-79.385296,Neighborhood


In [62]:
toronto_venues.replace('Neighborhood','Neighborhood_venue',inplace=True)
toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood_venue']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
315,The Beaches,43.676531,-79.295425,Upper Beaches,43.680563,-79.292869,Neighborhood_venue
388,Studio District,43.660629,-79.334855,Leslieville,43.662070,-79.337856,Neighborhood_venue
926,Central Bay Street,43.656091,-79.384930,Downtown Toronto,43.653232,-79.385296,Neighborhood_venue
1080,"Adelaide,King,Richmond",43.649700,-79.382582,Downtown Toronto,43.653232,-79.385296,Neighborhood_venue


In [63]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood_venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [64]:
toronto_onehot.shape

(2437, 262)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [65]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood_venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.00,0.030000,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,

#### Let's confirm the new size

In [66]:
toronto_grouped.shape

(98, 262)

**Remark**: The number of neighborhoods has decreased ==> for some Postcodes no venues were returned.

#### Let's print each neighborhood along with the top 5 most common venues

In [67]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.06
2        Hotel  0.05
3          Gym  0.03
4   Steakhouse  0.03


----Agincourt----
                venue  freq
0       Shopping Mall  0.14
1  Chinese Restaurant  0.14
2    Department Store  0.07
3                Park  0.07
4        Skating Rink  0.07


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                             venue  freq
0                 Sushi Restaurant   0.5
1                         Pharmacy   0.5
2  Molecular Gastronomy Restaurant   0.0
3               Neighborhood_venue   0.0
4                      Music Venue   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.13
1  Fried Chicken Joint  0.07
2                 Park  0.07
3           Beer Store  0.07
4          Coffee Shop  0.07


----Alderwood,Long Branch----
                   venue  

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Restaurant,Gym,American Restaurant,Bar,Burger Joint,Japanese Restaurant,Steakhouse
1,Agincourt,Chinese Restaurant,Shopping Mall,Sushi Restaurant,Supermarket,Bakery,Badminton Court,Pool,Bubble Tea Shop,Skating Rink,Vietnamese Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Sushi Restaurant,Pharmacy,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Liquor Store,Caribbean Restaurant,Fried Chicken Joint,Sandwich Place,Beer Store,Japanese Restaurant,Coffee Shop,Fast Food Restaurant
4,"Alderwood,Long Branch",Gym,Performing Arts Venue,Convenience Store,Pub,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


#### Last step: Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [98]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 3, 1, 4, 1, 1, 2, 1, 1, 2, 1, 1,
       1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3,
       0, 1, 0, 1, 1, 1, 2, 1, 0, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2,
       1, 2, 1, 1, 1, 1, 2, 1, 1, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [99]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

# Get rid of those neighborhoods not returning any venues, hence belong to no cluster
toronto_merged.dropna(inplace = True)

# Need to redjust type of label column
toronto_merged = toronto_merged.astype({'Cluster Labels': 'int32'})

toronto_merged #.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517,4,Home Service,Food,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725,1,Bar,Moving Target,Yoga Studio,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193,2,Construction & Landscaping,Gym / Fitness Center,Park,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
3,M1G,Scarborough,Woburn,43.768369,-79.217590,2,Park,Korean Restaurant,Coffee Shop,Business Service,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,0,Playground,Dry Cleaner,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750,1,Train Station,Women's Store,Restaurant,Indian Restaurant,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726276,-79.263625,1,Discount Store,Convenience Store,Department Store,Coffee Shop,Chinese Restaurant,Field,Fast Food Restaurant,Farmers Market,Fish & Chips Shop,Dumpling Restaurant
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713054,-79.285055,1,Bus Line,Bakery,Intersection,Coffee Shop,Bus Station,Soccer Field,Metro Station,Yoga Studio,Ethiopian Restaurant,Event Space
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.724235,-79.227925,1,Sandwich Place,Pharmacy,Bank,Liquor Store,Gift Shop,Bistro,Coffee Shop,Event Space,Eastern European Restaurant,Electronics Store
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696770,-79.259967,1,General Entertainment,Gym Pool,Skating Rink,Park,Gym,College Stadium,Farmers Market,Farm,Falafel Restaurant,Dog Run


Finally, let's visualize the resulting clusters

In [101]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

#### Cluster 1

In [102]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,0,Playground,Dry Cleaner,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
46,Central Toronto,0,Park,Playground,Garden,Gym Pool,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
48,Central Toronto,0,Playground,Convenience Store,Gym,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
50,Downtown Toronto,0,Playground,Grocery Store,Candy Store,Park,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop


#### Cluster 2

In [103]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,Bar,Moving Target,Yoga Studio,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
5,Scarborough,1,Train Station,Women's Store,Restaurant,Indian Restaurant,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
6,Scarborough,1,Discount Store,Convenience Store,Department Store,Coffee Shop,Chinese Restaurant,Field,Fast Food Restaurant,Farmers Market,Fish & Chips Shop,Dumpling Restaurant
7,Scarborough,1,Bus Line,Bakery,Intersection,Coffee Shop,Bus Station,Soccer Field,Metro Station,Yoga Studio,Ethiopian Restaurant,Event Space
8,Scarborough,1,Sandwich Place,Pharmacy,Bank,Liquor Store,Gift Shop,Bistro,Coffee Shop,Event Space,Eastern European Restaurant,Electronics Store
9,Scarborough,1,General Entertainment,Gym Pool,Skating Rink,Park,Gym,College Stadium,Farmers Market,Farm,Falafel Restaurant,Dog Run
12,Scarborough,1,Chinese Restaurant,Shopping Mall,Sushi Restaurant,Supermarket,Bakery,Badminton Court,Pool,Bubble Tea Shop,Skating Rink,Vietnamese Restaurant
13,Scarborough,1,Pizza Place,Pharmacy,Golf Course,Coffee Shop,Chinese Restaurant,Shopping Mall,Bus Stop,Fried Chicken Joint,Thai Restaurant,Hobby Shop
14,Scarborough,1,Sushi Restaurant,Pharmacy,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
15,Scarborough,1,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Thrift / Vintage Store,Pharmacy,Sandwich Place,Other Great Outdoors,Grocery Store,Coffee Shop,Camera Store


#### Cluster 3

In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,2,Construction & Landscaping,Gym / Fitness Center,Park,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
3,Scarborough,2,Park,Korean Restaurant,Coffee Shop,Business Service,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio
19,North York,2,Construction & Landscaping,Trail,Park,Dog Run,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
23,North York,2,Convenience Store,Park,Bank,Speakeasy,Cuban Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
25,North York,2,Food & Drink Shop,Park,Bus Stop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
26,North York,2,Construction & Landscaping,Park,Burger Joint,Gas Station,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
30,North York,2,Park,Airport,Food Court,Coffee Shop,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
34,North York,2,Park,Grocery Store,Yoga Studio,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Dry Cleaner
41,East Toronto,2,Park,Bus Line,Discount Store,Grocery Store,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Yoga Studio
59,Downtown Toronto,2,Park,Pier,Harbor / Marina,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


#### Cluster 4

In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,3,Gift Shop,Auto Garage,Construction & Landscaping,Bakery,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
11,Scarborough,3,Auto Garage,Yoga Studio,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Dry Cleaner
94,Etobicoke,3,Gift Shop,Donut Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### Cluster 5

In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Home Service,Food,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
32,North York,4,Home Service,Insurance Office,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Dry Cleaner
